In [1]:
import sqlite3
import pandas as pd

# Connect to Chinook database
conn = sqlite3.connect('chinook.db')

print("Connected to Chinook database")

Connected to Chinook database


In [2]:
# Configure pandas display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 30)

In [3]:
SCHEMA REFERENCE
Key Tables andColumns
tracks

TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer
Milliseconds, Bytes, UnitPrice

albums

AlbumId, Title, ArtistId

artists

ArtistId, Name

customers

CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId

invoices

InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total

invoice_items

InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity

genres

GenreId, Name

media_types

MediaTypeId, Name

employees

EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email

SyntaxError: invalid syntax (1543158634.py, line 1)

In [3]:
# Problem 1: Revenue per customer (Total money spent by each csutomer)

query = """
        SELECT Customer.FirstName, Customer.LastName, SUM(Invoice.Total) AS Revenue
        FROM Customer
        JOIN Invoice
        ON Customer.CustomerId = Invoice.CustomerId
        GROUP BY Customer.CustomerId, Customer.FirstName, Customer.LastName
        ORDER BY Revenue DESC;
        
        """

result = pd.read_sql_query(query, conn)
print(result)


    FirstName      LastName  Revenue
0      Helena          Holý    49.62
1     Richard    Cunningham    47.62
2        Luis         Rojas    46.62
3    Ladislav        Kovács    45.62
4        Hugh      O'Reilly    45.62
5       Frank       Ralston    43.62
6       Julia       Barnett    43.62
7        Fynn    Zimmermann    43.62
8      Astrid        Gruber    42.62
9      Victor       Stevens    42.62
10      Terhi    Hämäläinen    41.62
11  František   Wichterlová    40.62
12   Isabelle       Mercier    40.62
13   Johannes  Van der Berg    40.62
14       Luís     Gonçalves    39.62
15   François      Tremblay    39.62
16      Bjørn        Hansen    39.62
17       Jack         Smith    39.62
18        Dan        Miller    39.62
19    Heather       Leacock    39.62
20       João     Fernandes    39.62
21      Wyatt        Girard    39.62
22   Jennifer      Peterson    38.62
23        Tim         Goyer    38.62
24    Camille       Bernard    38.62
25  Dominique      Lefebvre    38.62
2

In [6]:
# Problem 2: Revenue per country (Total money spent)

query = """
        SELECT Customer.Country, SUM(Total) as Revenue
        FROM Customer
        JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
        GROUP BY Customer.Country
        ORDER BY Revenue DESC;
        
        
        """

result = pd.read_sql_query(query, conn)
print(result)

           Country  Revenue
0              USA   523.06
1           Canada   303.96
2           France   195.10
3           Brazil   190.10
4          Germany   156.48
5   United Kingdom   112.86
6   Czech Republic    90.24
7         Portugal    77.24
8            India    75.26
9            Chile    46.62
10         Ireland    45.62
11         Hungary    45.62
12         Austria    42.62
13         Finland    41.62
14     Netherlands    40.62
15          Norway    39.62
16          Sweden    38.62
17           Spain    37.62
18          Poland    37.62
19           Italy    37.62
20         Denmark    37.62
21         Belgium    37.62
22       Australia    37.62
23       Argentina    37.62


In [11]:
# Problem 3: Customers per city within each country

query = """
        SELECT Customer.City, Customer.Country, COUNT(Customer.CustomerId) AS Customer_Number
        FROM Customer
        GROUP BY Customer.City, Customer.Country;


        
        
        
        """

result = pd.read_sql_query(query, conn)
print(result)

                   City         Country  Customer_Number
0             Amsterdam     Netherlands                1
1             Bangalore           India                1
2                Berlin         Germany                2
3              Bordeaux          France                1
4                Boston             USA                1
5              Brasília          Brazil                1
6              Brussels         Belgium                1
7              Budapest         Hungary                1
8          Buenos Aires       Argentina                1
9               Chicago             USA                1
10           Copenhagen         Denmark                1
11            Cupertino             USA                1
12                Delhi           India                1
13                Dijon          France                1
14               Dublin         Ireland                1
15           Edinburgh   United Kingdom                1
16             Edmonton        

In [14]:
# Problem 4: Gneres with more than 100 tracks

query = """
        SELECT Genre.Name, COUNT(Track.TrackId) AS Track_Number
        FROM Genre
        JOIN Track ON Genre.GenreId = Track.GenreId
        GROUP BY Genre.GenreId, Genre.Name
        HAVING COUNT(Track.TrackId) > 100;


        
        
        
        """

result = pd.read_sql_query(query, conn)
print(result)

                 Name  Track_Number
0                Rock          1297
1                Jazz           130
2               Metal           374
3  Alternative & Punk           332
4               Latin           579


In [15]:
# Problem 5: Average track length per genre (Include genre name)

query = """
        SELECT Genre.Name, AVG(Track.Milliseconds) AS Avg_Track_length
        FROM Genre
        JOIN Track ON Genre.GenreId = Track.GenreId
        GROUP BY Genre.GenreId, Genre.Name;


        
        
        
        """

result = pd.read_sql_query(query, conn)
print(result)

                  Name  Avg_Track_length
0                 Rock      2.839100e+05
1                 Jazz      2.917554e+05
2                Metal      3.097494e+05
3   Alternative & Punk      2.343538e+05
4        Rock And Roll      1.346435e+05
5                Blues      2.703598e+05
6                Latin      2.328593e+05
7               Reggae      2.471778e+05
8                  Pop      2.290341e+05
9           Soundtrack      2.443709e+05
10          Bossa Nova      2.195900e+05
11      Easy Listening      1.891642e+05
12         Heavy Metal      2.974529e+05
13            R&B/Soul      2.200669e+05
14   Electronica/Dance      3.029858e+05
15               World      2.249238e+05
16         Hip Hop/Rap      1.781763e+05
17     Science Fiction      2.625549e+06
18            TV Shows      2.145041e+06
19    Sci Fi & Fantasy      2.911783e+06
20               Drama      2.575284e+06
21              Comedy      1.585264e+06
22         Alternative      2.640585e+05
23           Cla

In [19]:
# Problem 6: For each customer; number of invoices and total amount spent

query = """
        SELECT Customer.FirstName AS Customer, COUNT(Invoice.InvoiceId) AS Number_Of_Invoices, SUM(Invoice.Total) AS Total
        FROM Customer
        JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
        GROUP BY Customer.CustomerId, Customer.FirstName
        ORDER BY Total DESC;
        
        """

result = pd.read_sql_query(query, conn)
print(result)

     Customer  Number_Of_Invoices  Total
0      Helena                   7  49.62
1     Richard                   7  47.62
2        Luis                   7  46.62
3    Ladislav                   7  45.62
4        Hugh                   7  45.62
5       Frank                   7  43.62
6       Julia                   7  43.62
7        Fynn                   7  43.62
8      Astrid                   7  42.62
9      Victor                   7  42.62
10      Terhi                   7  41.62
11  František                   7  40.62
12   Isabelle                   7  40.62
13   Johannes                   7  40.62
14       Luís                   7  39.62
15   François                   7  39.62
16      Bjørn                   7  39.62
17       Jack                   7  39.62
18        Dan                   7  39.62
19    Heather                   7  39.62
20       João                   7  39.62
21      Wyatt                   7  39.62
22   Jennifer                   7  38.62
23        Tim   

In [20]:
# Problem 7: For each customer; number of invoices and total amount spent
# only include customers with more than 8 invoices

query = """
        SELECT Customer.FirstName AS Customer, COUNT(Invoice.InvoiceId) AS Number_Of_Invoices, SUM(Invoice.Total) AS Total
        FROM Customer
        JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId
        GROUP BY Customer.CustomerId, Customer.FirstName
        HAVING COUNT(Invoice.InvoiceId) > 8
        ORDER BY Total DESC;
        
        """

result = pd.read_sql_query(query, conn)
print(result)

Empty DataFrame
Columns: [Customer, Number_Of_Invoices, Total]
Index: []
